<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [73]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-02-27 12:36:44--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  13.7MB/s    in 1m 56s  

2021-02-27 12:38:40 (12.0 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [74]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [5]:
word = 'dog'
wv_embeddings.similar_by_word(word=word, topn=5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

In [6]:
word = 'cat'
wv_embeddings.similar_by_word(word=word, topn=5)

[('-vet', 0.7583354711532593),
 ('dog', 0.6852341294288635),
 ('/proc/version', 0.681588888168335),
 ('feline', 0.6779966354370117),
 ('nyan', 0.6685197353363037)]

Входит слово 'cats', но не 'cat'. А вот слово 'dog' входит в топ 5 для близких к слову 'cat'.

In [7]:
wv_embeddings.most_similar(positive=['dog', 'cat'], topn=5)

[('feline', 0.7719643115997314),
 ('cats', 0.7606382369995117),
 ('animal', 0.7583404779434204),
 ('meow', 0.7338752746582031),
 ('dogs', 0.7305051684379578)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [93]:
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import StandardScaler as SS
import numpy as np

tokenizer = WordPunctTokenizer()

In [114]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    q = question #.lower()
    tokens = tokenizer.tokenize(q)
    n_words = 0
    vector = np.zeros(shape=dim)
    for word in tokens:
        if word in embeddings:
           vector += embeddings[word]
           n_words += 1
    vector = vector if n_words==0 else vector / n_words
    return vector.reshape(1, -1)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [97]:
question = 'I love neural networks'
sec_symbol = question_to_vec(question, wv_embeddings, tokenizer)[2]
print(round(sec_symbol, 2))

-0.43


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [3]:
def _bool(x):
  return 1 if x >= 0 else 0

In [4]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    dup_ranks = np.array(dup_ranks)
    hits_value =  np.array([*map(_bool, k - dup_ranks)]).sum() / len(dup_ranks)
    
    return hits_value 

In [5]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dup_ranks = np.array(dup_ranks)
    bool_ = np.array([*map(_bool, k - dup_ranks)])
    dcg_value = 1 / np.log2(1 + dup_ranks)
    dcg_value = dcg_value @ bool_.T / len(dup_ranks)

    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [6]:
import pandas as pd

In [9]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [21]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [10]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [11]:
!unzip -q /content/gdrive/MyDrive/stackoverflow_similar_questions.zip

Считайте данные.

In [12]:
import re
import numpy as np
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

In [63]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        Quest = line.split('\t')
        #quest = [*map(lambda x: x.lower(), Quest)]
        data.append(Quest)
    return data

Нам понадобиться только файл validation.

In [64]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [66]:
len(validation_data)

3760

Размер нескольких первых строк

In [67]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [68]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [69]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    main_quest = question_to_vec(question, embeddings=embeddings, tokenizer=tokenizer, dim=dim).reshape(1, -1)
    
    dist = []
    for i, quest in enumerate(candidates): 
        q_emb = question_to_vec(quest, embeddings=embeddings, tokenizer=tokenizer, dim=dim).reshape(1, -1)

        if q_emb.shape[1] == 200:
           d = cosine_similarity(main_quest, q_emb)
           dist.append((i, float(d)))

    dist.sort(reverse=True, key=lambda x: x[1])

    results = []
    for i, _ in dist:
        results.append((i, candidates[i]))
    return results

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [19]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [54]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'),     #скрыт
            (*, 'select2 not displaying search results'),                  #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

SyntaxError: ignored

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: 0, 2, 1

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [70]:
from tqdm.notebook import tqdm

In [83]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data[:1000])):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [84]:
len(wv_ranking)

1000

In [85]:
for k in tqdm([1, 5, 10, 100, 500, 750, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.386
DCG@  10: 0.349 | Hits@  10: 0.438
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.875
DCG@ 750: 0.427 | Hits@ 750: 0.929
DCG@1000: 0.434 | Hits@1000: 1.000



### Эмбеддинги, обученные на корпусе похожих вопросов

In [117]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [118]:
from time import time 

In [119]:
%%time
def glu(x):
    sentence = x[0]+ ' ' + x[1]
    return  tokenizer.tokenize(sentence)
    
words = [*map(glu, train_data)]

CPU times: user 14.7 s, sys: 2.7 s, total: 17.4 s
Wall time: 18.3 s


In [90]:
%%time
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,       # data for model to train on
                              size=200,    # embedding vector size
                              min_count=5, # consider words that occured at least 5 times
                              window=10).wv

CPU times: user 6min 41s, sys: 1.4 s, total: 6min 42s
Wall time: 3min 35s


In [115]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data[:1000])):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [116]:
for k in tqdm([1, 5, 10, 100, 500, 750, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.282 | Hits@   1: 0.282
DCG@   5: 0.349 | Hits@   5: 0.408
DCG@  10: 0.370 | Hits@  10: 0.474
DCG@ 100: 0.421 | Hits@ 100: 0.731
DCG@ 500: 0.447 | Hits@ 500: 0.927
DCG@ 750: 0.452 | Hits@ 750: 0.974
DCG@1000: 0.454 | Hits@1000: 1.000



### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Лучше использовать токенайзер, он позволяет более грамотно разделить предложения на слова, что увеличивает качесто модели. Также еще можноиспользовать различные методы предобработки, например, стемминг или лемматизацию, также можно убрать все знаки припенания. Однако с предобработкой необходимо быть осторожным, поскольку она может и ухудшить качество. Так в моем случае лемматизация, примененная к train выборке ухудшала качество тестов. 

Как видно из опыта лучшее качество показывают свои собственные эмбеддинги, тк они позволяют обучиться именно под конкретную задачу. Но в сулчае, если провести дополнительно понижение регистра и предобработку, предобученные  эмбеддинги показывают более высокий результат. В случае же, если сильную предобработку текстов не проводить, то самомтоятельная модель дает большее качество. Это связано в первую очередь с тем, что предобучение позволяет унифицировать слова, в таком случае мы избавляемся от уникальных особенностей именно нашей выборки и предобученная модель (тк она была обучена на большем количестве слов) дает теперь более качественный результат.

Для улучшения качество модели можно попробовать дополнительно поиграться с гиперпараметрами, добадивть (аккуратно) стемминг или лемматизацию а также попытаться по-разному тренировать модель, например, не на парах вопросов, а на всем массиве вопросов в целом.
